In [15]:
import os
import tensorflow as tf
import keras
from IPython.display import Image, display
from IPython.core.display import HTML
import requests

In [16]:
print(keras.__version__, tf.__version__)

2.2.4 1.13.1


In [3]:
### Load a pretrained inception_v3
inception_model = keras.applications.inception_v3.InceptionV3(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# Define a destination path for the model
MODEL_EXPORT_DIR = r'C:\tmp\inception_v3'
MODEL_VERSION = 1
MODEL_EXPORT_PATH = os.path.join(MODEL_EXPORT_DIR, str(MODEL_VERSION))
print("Model dir: ", MODEL_EXPORT_PATH)

Model dir:  C:\tmp\inception_v3\1


In [5]:
# Before saving the model, let's print the input tensors of the model:
print(inception_model.inputs)
print(inception_model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 299, 299, 3) dtype=float32>]
[<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>]


In [6]:
# We'll need to create an input mapping, and name each of the input tensors.
# In the inception_v3 Keras model, there is only a single input and we'll name it 'image'
input_names = ['image']
name_to_input = {name: t_input for name, t_input in zip(input_names, inception_model.inputs)}
print(name_to_input)

{'image': <tf.Tensor 'input_1:0' shape=(?, 299, 299, 3) dtype=float32>}


In [18]:
# Save the model to the MODEL_EXPORT_PATH
# Note using 'name_to_input' mapping, the names defined here will also be used for querying the service later
tf.saved_model.simple_save(
    keras.backend.get_session(),
    MODEL_EXPORT_PATH,
    inputs=name_to_input,
    outputs={t.name: t for t in inception_model.outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: C:\tmp\inception_v3\1\saved_model.pb


In [7]:
ls C:\tmp\inception_v3\1

 Volume in drive C is 系统
 Volume Serial Number is 0006-8F84

 Directory of C:\tmp\inception_v3\1

08/28/2019  03:58 PM    <DIR>          .
08/28/2019  03:58 PM    <DIR>          ..
08/28/2019  03:58 PM         3,432,445 saved_model.pb
08/28/2019  03:57 PM    <DIR>          variables
               1 File(s)      3,432,445 bytes
               3 Dir(s)   9,601,069,056 bytes free


1. Copy the saved model to the hosts' specified directory. (source=/tmp/inception_v3 in this example)
   Run the docker:

2. Run the docker:

docker run -d -p 8501:8501 --name keras_inception_v3 --mount type=bind,source=C:\tmp\inception_v3,target=/models/inception_v3 -e MODEL_NAME=inception_v3 -t tensorflow/serving

3. Verify that there's network access to the Tensorflow service. In order to get the local docker ip (172.*.*.*) for testing run:

docker inspect keras_inception_v3

"HostIp": "0.0.0.0" , "HostPort": "8501"
    
"IPAddress": "172.17.0.2"

!curl {HOST_IP}:{HOST_PORT}

curl  172.17.0.2:8501

# 3.Query the model with an HTTP client and verify the same output as the local model
### Download a Dog (Tibetan Mastiff) and a Cat (Ragdol) image for our tests

In [32]:
def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()
#     print(response.content)

    with open(filename, 'wb') as f:
        f.write(response.content)

cat_url = "https://upload.wikimedia.org/wikipedia/commons/c/c0/Ragdoll_Blue_Colourpoint.jpg"
dog_url = "https://3milliondogs.com/blog-assets-two/2015/06/11407122_926045227460335_778769622795895821_n.jpg"

cat_filename = os.path.join("/tmp", "cat.jpg")
dog_filename = os.path.join("/tmp", "dog.jpg")

# download_file(cat_url, cat_filename)
# download_file(dog_url, dog_filename)

### Define a simple method to query the model locally

In [17]:
import requests
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.inception_v3 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions

INCEPTIONV3_TARGET_SIZE = (299, 299)

def predict(image_path):
    x = img_to_array(load_img(image_path, target_size=INCEPTIONV3_TARGET_SIZE))
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return inception_model.predict(x)

### Define a client class for Tensorflow Serving

In [18]:
# Define a Base client class for Tensorflow Serving
class TFServingClient:
    """
    This is a base class that implements a Tensorflow Serving client
    """
    TF_SERVING_URL_FORMAT = '{protocol}://{hostname}:{port}/v1/models/{endpoint}:predict'

    def __init__(self, hostname, port, endpoint, protocol="http"):
        self.protocol = protocol
        self.hostname = hostname
        self.port = port
        self.endpoint = endpoint

    def _query_service(self, req_json):
        """

        :param req_json: dict (as define in https://cloud.google.com/ml-engine/docs/v1/predict-request)
        :return: dict
        """
        server_url = self.TF_SERVING_URL_FORMAT.format(protocol=self.protocol,
                                                       hostname=self.hostname,
                                                       port=self.port,
                                                       endpoint=self.endpoint)
        response = requests.post(server_url, json=req_json)
        response.raise_for_status()
        return np.array(response.json()['predictions'])


# Define a specific client for our inception_v3 model
class InceptionV3Client(TFServingClient):
    # INPUT_NAME is the config value we used when saving the model (the only value in the `input_names` list)
    INPUT_NAME = "image"
    TARGET_SIZE = INCEPTIONV3_TARGET_SIZE

    def load_image(self, image_path):
        """Load an image from path"""
        img = img_to_array(load_img(image_path, target_size=self.TARGET_SIZE))
        return preprocess_input(img)

    def predict(self, image_paths):
        imgs = [self.load_image(image_path) for image_path in image_paths]

        # Create a request json dict
        req_json = {
            "instances": [{self.INPUT_NAME: img.tolist()} for img in imgs]
        }
        print(req_json)
        return self._query_service(req_json)

In [22]:
# Instantiate a client
hostname = "172.17.0.1"
port = "8501"
endpoint="inception_v3"
client = InceptionV3Client(hostname=hostname, port=port, endpoint=endpoint)

# Validate Results

In [28]:
cat_url = "https://upload.wikimedia.org/wikipedia/commons/c/c0/Ragdoll_Blue_Colourpoint.jpg"
dog_url = "https://3milliondogs.com/blog-assets-two/2015/06/11407122_926045227460335_778769622795895821_n.jpg"

cat_filename = os.path.join("/tmp", "cat.jpg")
dog_filename = os.path.join("/tmp", "dog.jpg")

cat_local_preds = predict(cat_filename)
cat_local_preds

array([[1.35093549e-04, 1.42409757e-04, 1.32260568e-04, 2.68966047e-04,
        2.77036190e-04, 9.40881437e-05, 1.02783539e-04, 5.49721663e-05,
        8.26971082e-05, 1.82478660e-04, 1.99738352e-04, 3.53201380e-04,
        1.10239002e-04, 1.39760159e-04, 7.37417577e-05, 1.84919540e-04,
        2.67350202e-04, 1.57514820e-04, 1.09088694e-04, 1.82222153e-04,
        1.03178842e-04, 1.79856506e-04, 4.66891688e-05, 1.05660314e-04,
        1.30639542e-04, 4.01480065e-04, 3.31514631e-04, 1.50513384e-04,
        1.46106191e-04, 1.86195335e-04, 1.37730574e-04, 1.72592263e-04,
        9.05942070e-05, 1.50792417e-04, 1.30152810e-04, 5.28882316e-04,
        1.86425605e-04, 1.37430223e-04, 8.96889105e-05, 4.42510682e-05,
        1.38400108e-04, 1.65416539e-04, 1.16504561e-04, 6.61625454e-05,
        8.09565754e-05, 1.02189682e-04, 1.01156940e-04, 1.05350016e-04,
        8.01079295e-05, 1.94227701e-04, 1.61176373e-04, 1.69345411e-04,
        1.60607713e-04, 2.58093758e-04, 4.46194026e-04, 8.636467

In [23]:
cat_remote_preds = client.predict([cat_filename])
cat_remote_preds

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [ ]:
# Validate the prediction values are the same for the local and remote models
assert np.allclose(cat_local_preds, cat_remote_preds, atol=1e-07)
# assert np.allclose(dog_local_preds, dog_remote_preds, atol=1e-07)

# Show model predictions

In [29]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image


def grid_iterator(n_rows, n_cols, axis_size=2.5, total=None, fig_title=None):
    """
    matplotlib helper method for plotting a grid of graphs


    :param n_rows: int
    :param n_cols: int
    :param axis_size: float
    :param total: int (stopping condition - stop before n_rows * n_cols subplots are filled in the grid)
    :param fig_title: str
    """
    fig = plt.figure()
    gs = gridspec.GridSpec(n_rows, n_cols, top=1., bottom=0., right=1., left=0., hspace=0.15, wspace=0.1)

    index = 0
    for r in range(n_rows):
        for c in range(n_cols):
            ax = fig.add_subplot(gs[r, c])
            yield index, ax

            index += 1
            # Allowing for not filling all spots
            if total and index >= total:
                break

    width, height = n_cols * axis_size, n_rows * axis_size
    fig.set_size_inches(width, height)

    # Add title to the figure
    if fig_title is not None:
        fig.suptitle(fig_title, fontsize=16, y=1.05)


def get_titles(predicts):
    """
    Helper method that concats the highest classification with all classification that have a score >=0.1
    """
    titles = []
    for decoded_pred in decode_predictions(predicts):
        res = []
        for _, cls, score in decoded_pred:
            if score >= 0.1 or not res:
                res.append("%s(%.2f)" % (cls, score))

        title = ",".join(res)
        titles.append(title)
    return titles

In [33]:
# Define a list of image urls
image_urls = [
    dog_url,
    cat_url]

# Download files
img_files = []
for i, image_url in enumerate(image_urls):
    filename = os.path.join("/tmp", "%d.jpg" % i)
    download_file(image_url, filename)
    img_files.append(filename)

In [43]:
img_files = ['C:\tmp\0.jpg']

In [44]:
# Run predictions
# %time predicts = client.predict(img_files)
predicts = predict(img_files)
# assert len(predicts) == len(img_files)

AttributeError: 'list' object has no attribute 'read'

In [ ]:
titles = get_titles(predicts)

for i, ax in grid_iterator(n_rows=5, n_cols=3, axis_size=4, total=len(img_files)):
    title = titles[i]
    img = Image.open(img_files[i])
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('auto')
    ax.set_title(title)